In [2]:
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [5]:
test.head()

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
1,348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
2,348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
3,348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
4,348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


In [6]:
def smape(y_pred, y_true):
    assert(len(y_pred) == len(y_true))
    
    numerator = abs(y_pred - y_true)
    denomerator = (abs(y_pred) + abs(y_true)) / 2
    ans = (100.0 / len(y_pred)) * np.sum(numerator / denomerator)
    return ans

In [7]:
frac = 0.4

train_sampled = train.sample(frac=frac)

print(len(train_sampled))

28983


In [8]:
only_3_shift = train_sampled[train_sampled['shift'] == 3]
only_not_3_shift = train_sampled[train_sampled['shift'] != 3]
print(len(only_3_shift), len(only_not_3_shift))

(9689, 19294)


In [11]:
only_3_shift_w_mean = only_3_shift.copy()
only_3_shift_w_mean['mean'] = only_3_shift[only_3_shift.columns.values[6:]].mean(axis=1)

In [12]:
only_3_shift_w_mean.head()

,Num,y,year,week,shift,item_id,f1,f2,f3,f4,...,f52,f53,f54,f55,f56,f57,f58,f59,f60,mean
26860,130900,179469,2013,40,3,20438711,81696.0,131539.0,70345.0,89040.0,...,112140.0,123525.0,120429.0,110120.0,111974.0,118976.0,126642.0,99802.0,109937.0,105445.066667
17209,83328,26194,2013,26,3,20437676,8812.0,9145.0,13155.0,6400.0,...,8787.0,8780.0,12558.0,11190.0,8785.0,14320.0,14620.0,17540.0,10900.0,10730.066667
30732,150831,61627,2013,46,3,20453393,24438.0,6998.0,8278.0,9652.0,...,111902.0,61616.0,65226.0,53015.0,71247.0,110784.0,54229.0,47362.0,42126.0,39804.566667
35505,174280,234908,2014,1,3,20435814,163767.0,243118.0,144349.0,123366.0,...,138331.0,139847.0,196333.0,79785.0,144177.0,144207.0,187463.0,102670.0,133635.0,161835.300000
11726,55847,6314,2013,18,3,20439405,3940.0,3760.0,2095.0,2092.0,...,3245.0,3483.0,3087.0,2490.0,1910.0,4415.0,3765.0,3428.0,3180.0,2891.166667


In [14]:
%%time

frac = 0.85
valid = train.sample(frac=frac)

valid_3_shift = valid[valid['shift'] == 3]
valid_others = valid[valid['shift'] != 3]

print(len(valid_3_shift), len(valid_others))

valid_3_shift_w_mean = valid_3_shift.copy()
valid_3_shift_w_mean['mean'] = valid_3_shift[valid_3_shift.columns.values[6:]].mean(axis=1)

valid_others_w_mean = valid_3_shift.copy()
valid_others_w_mean['mean'] = np.exp(valid_others_w_mean[valid_others_w_mean.columns.values[6:]].mean(axis=1))
valid_others_w_mean['y'] = np.sqrt(valid_others_w_mean['y'])

(20210, 41378)
Wall time: 101 ms


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: overflow encountered in exp


In [16]:
%%time

model_xg = xgboost.XGBRegressor(n_estimators=150, max_depth=20)
model_xg.fit(only_3_shift_w_mean.drop(['Num', 'y'], axis=1), only_3_shift_w_mean['y'])

Wall time: 17.9 s


In [17]:
smape(model_xg.predict(valid_3_shift_w_mean.drop(['Num', 'y'], axis=1)), valid_3_shift_w_mean['y'])

15.539817753432706

In [18]:
others = only_not_3_shift.copy()
others['mean'] = np.exp(others[others.columns.values[6:]].mean(axis=1))
others['y'] = np.sqrt(others['y'])

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


In [19]:
model_xg_other = xgboost.XGBRegressor(n_estimators=150, max_depth=20)
model_xg_other.fit(others.drop(['Num', 'y'], axis=1), others['y'])

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [20]:
smape(model_xg_other.predict(valid_others_w_mean.drop(['Num', 'y'], axis=1)), valid_others_w_mean['y'])

14.77737120322563

In [21]:
test.head()

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
1,348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
2,348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
3,348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
4,348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


In [22]:
valid_mean = valid.copy()
valid_mean['mean'] = valid_mean[valid_mean.columns.values[6:]].mean(axis=1)

In [23]:
smape(model_xg.predict(valid_mean.drop(['Num', 'y'], axis=1)), valid_mean['y'])

25.808339033172047

In [24]:
smape(model_xg_other.predict(valid_mean.drop(['Num', 'y'], axis=1)), valid_mean['y'])

187.65682590358512

In [25]:
test_mean = test.copy()
test_mean['mean'] = test_mean[test_mean.columns.values[6:]].mean(axis=1)

In [26]:
preds = model_xg.predict(test_mean.drop(['Num'], axis=1))

In [27]:
sample_submission['y'] = preds

In [29]:
sample_submission.head(10)

,Num,y
0,348622,1044.086914
1,348623,26465.800781
2,348624,367160.250000
3,348625,26096.832031
4,348626,37.358944
5,348627,129013.039062
6,348628,81602.250000
7,348629,94526.218750
8,348630,98701.531250
9,348631,3183.448730


In [30]:
print sample_submission[sample_submission['y'] < 0]

Empty DataFrame
Columns: [Num, y]
Index: []


In [31]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [33]:
sample_submission.to_csv("answer.tsv", sep=',', index=False)